<a href="https://colab.research.google.com/github/shivam-kotwalia/data-science-project/blob/master/Tensorflow%20Serving%20on%20Google%20Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [2]:
import tensorflow as tf
tf.__version__

'2.2.0-rc2'

In [0]:
from tensorflow.keras.datasets import fashion_mnist

In [4]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [0]:
from tensorflow.keras.models      import Sequential
from tensorflow.keras.layers      import Flatten, Dense, Conv2D
from tensorflow.keras.losses      import categorical_crossentropy
from tensorflow.keras.optimizers  import SGD
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.utils       import to_categorical

In [0]:
model = Sequential()
model.add(Conv2D(8, kernel_size=(3,3), strides=(1,1), input_shape=(28, 28, 1)))
model.add(Flatten())
model.add(Dense(256, activation=relu))
model.add(Dense(10, activation=softmax))

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               200960    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile("rmsprop", categorical_crossentropy, metrics=["acc"])

In [0]:
x_train = x_train/255.0
x_test = x_test/255.0

In [0]:
y_train = to_categorical(y_train, num_classes=10)
y_test  = to_categorical(y_test,  num_classes=10)

In [43]:
x_train.shape, y_train.shape

((60000, 28, 28), (60000, 10))

In [48]:
model.fit(x_train, y_train, epochs=20, validation_split=0.2)

Epoch 1/20


TypeError: ignored